# Introduction
[Spartan](https://www.cs.nyu.edu/web/Research/Theses/power_russell.pdf) とはArray式言語の並列処理用のフレームワークです。

* 今回はPythonの高速化のために使用してみました。

![](./pictures/Screen Shot 2015-11-16 at 10.34.13.jpg)

* 今回はシングルPCで動作を確認します。

Spartanの簡単な構成を説明しておきます。

* Array-language frontend
 * NmyPy array言語をユーザーは使用可能。
* Intermediate representation
 * prefix scans, slicing and filteringなどのarray操作が可能
* Optimization
 * 代数的なルールの書き換えを行い、必要のない一時的な要素を排除する。
* Distributed array backend
 * 多次元配列での計算

以下では、このSpartanを利用し実際に動作させる所まで解説します。

1. [ライブラリ導入](#ライブラリ導入) 
2. [初期設定](#初期設定) 
3. [サーバー設定](#サーバー設定)
4. [線形回帰処理](#線形回帰処理) 

日本語の解説ページは下記。

[Python　分散処理　spartan](http://qiita.com/GushiSnow/private/05f3c5632cbed6e57900)



## 1.ライブラリ導入

Spartanのライブラリを導入します。



In [4]:
import spartan as sp


## 2.初期設定

下記を設定しています。
* Spartanの初期化：sp.initialize()
* 次元数の設定：N_DIM = 10
* サンプル数の設定：N_EXAMPLES = 1000 * 1000
* 重み更新度合い：EPSILON 
* xを10次元のデータで生成
* yをサンプル数生成


In [5]:
sp.initialize()

N_DIM = 10
N_EXAMPLES = 1000 * 1000
EPSILON = 1e-6

x = 100 * sp.ones((N_EXAMPLES, N_DIM)) + sp.rand(N_EXAMPLES, N_DIM)
y = sp.ones((N_EXAMPLES, 1))

## 3.#サーバー設定

1つのサーバーに対して10次元の重みを処理するように設定

* 初期値は乱数で設定。


In [6]:
# put weights on one server
w = sp.rand(N_DIM, 1)

## 4.線形回帰処理

単純な線形回帰の処理を行なっています。

* 入力xと重みwによって値yを出力します。
* 値の差分を取得します。
* 差分を重み更新の値に変換
* 重みを更新します。更新率をかけることとサンプル数で割ることによって更新しすぎないように調整しています。



In [5]:
for i in range(50):
    yp = sp.dot(x, w)
    diff = x * (yp - y)
    grad = sp.sum(diff, axis=0).reshape((N_DIM, 1))
    w = w - (grad / N_EXAMPLES * EPSILON)
    print grad.sum().glom()